In [2]:
import pandas as pd
import numpy as np

Car data integrate

In [69]:
travel = pd.read_csv("./data/df_travel.csv")
car = pd.read_csv("./data/df_car.csv")

In [13]:
travel.groupby("Transportation type").size()

Transportation type
Car      23
Ferry     1
Plane    75
Train    37
dtype: int64

In [4]:
def transportationSplit(trans):
    if trans == "Car" or trans == "Car rental" or trans == "Bus" or trans == "Subway":
        return "Car"
    elif trans == "Plane" or trans == "Airplane" or trans == "Flight":
        return "Plane"
    else:
        return trans

In [70]:
travel["Transportation type"] = travel["Transportation type"].apply(transportationSplit)

In [ ]:
car_ids = car['car_id'].tolist()

# Define a function that returns a random car_id if Transportation type is 'Car'
def get_random_car_id(transport_type):
    if transport_type == 'Car':
        return int(np.random.choice(car_ids))
    else:
        return None

# Apply this function to the 'Transportation type' column and create a new 'car_id' column
travel['car_id'] = travel['Transportation type'].apply(get_random_car_id).astype('Int64')
#sum(travel.car_id.isna())
travel[~travel["car_id"].isnull()]

Flight Integration

In [72]:
flight = pd.read_csv("./data/df_flight.csv")

In [21]:
travel[travel["Transportation type"] == "Plane"].groupby("Destination_Country").size().index

Index(['Aus', 'Australia', 'Brazil', 'Cambodia', 'Cameroon', 'Canada',
       'France', 'Germany', 'Greece', 'Hawaii', 'Indonesia', 'Italy', 'Japan',
       'Mexico', 'Morocco', 'Netherlands', 'South Africa', 'Spain', 'Thai',
       'Thailand', 'UK', 'USA', 'United Arab Emirates'],
      dtype='object', name='Destination_Country')

In [22]:
flight.groupby("GEO Region").size().index

Index(['Asia', 'Australia / Oceania', 'Canada', 'Central America', 'Europe',
       'Mexico', 'Middle East', 'South America', 'US'],
      dtype='object', name='GEO Region')

In [73]:
def get_geo_region_and_flight_id(destination_country):
    country_to_region = {
        'Aus': 'Australia / Oceania',
        'Australia': 'Australia / Oceania',
        'Brazil': 'South America',
        'Cambodia': 'Asia',
        'Cameroon': 'Africa',  # Africa is not in geo regions list.
        'Canada': 'Canada',
        'France': 'Europe',
        'Germany': 'Europe',
        'Greece': 'Europe',
        'Hawaii': 'US', 
        'Indonesia': 'Asia',
        'Italy': 'Europe',
        'Japan': 'Asia',
        'Mexico': 'Mexico',
        'Morocco': 'Middle East',  
        'Netherlands': 'Europe',
        'South Africa': 'Africa',  # Africa is not in geo regions list.
        'Spain': 'Europe',
        'Thai': 'Asia', 
        'Thailand': 'Asia',
        'UK': 'Europe',
        'USA': 'US',
        'United Arab Emirates': 'Middle East'
    }
    
    region = country_to_region.get(destination_country, 'Unknown')
    if region == 'Unknown':
        return None
    region_flights = flight[flight['GEO Region'] == region]
    if not region_flights.empty:
        return np.random.choice(region_flights['flight_id'])
    else:
        return None


In [74]:
travel["flight_id"] = travel[travel["Transportation type"] == "Plane"]["Destination_Country"].apply(get_geo_region_and_flight_id).astype('Int64')

In [ ]:
travel[(travel["Transportation type"] == "Plane") & (travel["flight_id"].isna())]

Airbnb & Hotel Integrate

In [91]:
airbnb = pd.read_csv("./data/df_airbnb.csv")
hotel = pd.read_csv("./data/df_hotel.csv")

In [130]:
def accommodationSplit(type):
    if type == "Airbnb" or type == "Vacation rental" or type == "Guesthouse" or type == "Villa" or type == "Riad":
        return "Airbnb"
    elif type == "Hotel" or type == "Hostel" or type == "Resort":
        return "Hotel"

In [131]:
travel["Accommodation type"] = travel["Accommodation type"].apply(accommodationSplit)

In [133]:
def get_random_hotel_id(destination_city):
    if not isinstance(destination_city, str):
        return None
    matching_hotels = hotel[hotel['city'] == destination_city]
    if not matching_hotels.empty:
        return np.random.choice(matching_hotels['hotel_id'].values)
    else:
        return None


In [134]:
travel['hotel_id'] = travel.apply(
    lambda row: get_random_hotel_id(row['Destination_City']) if row["Accommodation type"] == "Hotel" else None,
    axis=1
).astype("Int64")

In [148]:
def get_random_airbnb_id(destination_city, destination_country):
    if not isinstance(destination_city, str):
        return None
    matching_airbnbs = airbnb[airbnb['City/State'] == destination_city]
    if not matching_airbnbs.empty:
        return np.random.choice(matching_airbnbs['airbnb_id'].values)
    else:
        matching_airbnbs = airbnb[airbnb['Country'] == destination_country]
        if not matching_airbnbs.empty:
            return np.random.choice(matching_airbnbs['airbnb_id'].values)
        else:
            return None

In [149]:
travel['airbnb_id'] = travel.apply(
    lambda row: get_random_airbnb_id(row['Destination_City'], row["Destination_Country"]) if row["Accommodation type"] == "Airbnb" else None,
    axis=1
).astype("Int64")

Travel Cleaning

In [3]:
def numberCleaning(num):
    if isinstance(num, str):
        num = num.replace(',', '')
        num = num.replace(' ', '')
        num = num.replace('USD', '')
    return num

In [85]:
travel["Accommodation cost"] = travel["Accommodation cost"].apply(numberCleaning)
travel["Transportation cost"] = travel["Transportation cost"].apply(numberCleaning)

In [150]:
travel.to_csv("./data/df_integrated_travel.csv", index=False)

In [8]:
df_travel = pd.read_csv("./data/df_integrated_travel.csv")
df_travel_new = pd.read_csv("./data/df_travel_new.csv")

In [9]:
df_travel_new["Accommodation cost"] = df_travel_new["Accommodation cost"].apply(numberCleaning)
df_travel_new["Transportation cost"] = df_travel_new["Transportation cost"].apply(numberCleaning)

In [5]:
df_travel_new = pd.concat([df_travel_new, df_travel[['car_id', 'flight_id', 'hotel_id', 'airbnb_id']]], axis=1)
df_travel_new['car_id'] = df_travel_new['car_id'].astype('Int64')
df_travel_new['flight_id'] = df_travel_new['flight_id'].astype('Int64')
df_travel_new['hotel_id'] = df_travel_new['hotel_id'].astype('Int64')
df_travel_new['airbnb_id'] = df_travel_new['airbnb_id'].astype('Int64')


In [7]:
df_travel_new.to_csv("./data/df_integrated_travel_new.csv", index=False)

In [3]:
df = pd.read_csv("./data/df_integrated_travel_new.csv")

In [5]:
df["Transportation type"] = df["Transportation type"].apply(transportationSplit)

In [7]:
df['car_id'] = df['car_id'].astype('Int64')
df['flight_id'] = df['flight_id'].astype('Int64')
df['hotel_id'] = df['hotel_id'].astype('Int64')
df['airbnb_id'] = df['airbnb_id'].astype('Int64')
df['Transportation cost'] = df['Transportation cost'].astype('Int64')
df.to_csv("./data/df_integrated_travel_new.csv", index=False)